In [2]:
!pip install --upgrade transformers

Requirement already up-to-date: transformers in c:\users\iyeng\anaconda3\lib\site-packages (4.11.3)


In [68]:
!pip install sumy

Processing c:\users\iyeng\appdata\local\pip\cache\wheels\56\ea\58\ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c\docopt-0.6.2-py2.py3-none-any.whl
Processing c:\users\iyeng\appdata\local\pip\cache\wheels\09\eb\0d\4ee773c6a4aadc2a43cb5c1d07f268f13c4cdc0eec88e7c1ef\pycountry-20.7.3-py2.py3-none-any.whl
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21692 sha256=32a7f8b574b306f561452bc1090260aac00656584af31c2bf08e7d5fc5d1651b
  Stored in directory: c:\users\iyeng\appdata\local\pip\cache\wheels\5f\0d\0c\2062d8c1758b4b1a2e42b4a63e6660d9ec2ba9463cfee9eeab
Successfully built breadability


In [209]:
!pip install spaCy

In [214]:
!python -m spacy download en

[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2021-10-23 02:54:11.208438: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-10-23 02:54:11.208874: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [210]:
import numpy as np
import pandas as pd
import transformers  # for abstractive summarization
import re
import sumy  # for extractive summarization
import nltk
import time
import csv
import spacy

In [78]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\iyeng\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [5]:
review_df = pd.read_csv("reviewSelected100.csv")
review_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,8aoJJdKEO3ypoZNszpPu7Q,bGgAL09pxLnV_FFgR4ZADg,ZBE-H_aUlicix_9vUGQPIQ,5,0,0,0,We had my Mother's Birthday Party here on 10/2...,2016-11-09 20:07:25
1,J5NOCLdhuhor7USRhtYZ8w,pFCb-1j6oI3TDjr26h2cJQ,e-YnECeZNt8ngm0tu4X9mQ,4,0,0,0,Good Korean grill near Eaton Centre. The marin...,2015-12-05 05:06:43
2,PXiLWAYRt3xnHaJ8MB4rzw,mEzc6LeTNiQgIVsq3poMbg,j7HO1YeMQGYo3KibMXZ5vg,5,2,1,3,Was recommended to try this place by few peopl...,2014-10-11 05:16:15
3,VrLarvxZYJm74yAqtpe9PQ,o-zUN2WEZgjQS7jnNsec0g,7e3PZzUpG5FYOTGt3O3ePA,3,0,0,0,Ambience: Would not expect something this nice...,2016-07-25 03:45:26
4,C1CUpidlVFprUCkApqzCmA,Wlx0iBXJvk4x0EeOt2Bz1Q,vuHzLZ7nAeT-EiecOkS5Og,1,11,0,3,Absolutely the WORST pool company that I have ...,2016-04-11 18:49:11


In [6]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15300 entries, 0 to 15299
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review_id    15300 non-null  object
 1   user_id      15300 non-null  object
 2   business_id  15300 non-null  object
 3   stars        15300 non-null  int64 
 4   useful       15300 non-null  int64 
 5   funny        15300 non-null  int64 
 6   cool         15300 non-null  int64 
 7   text         15300 non-null  object
 8   date         15300 non-null  object
dtypes: int64(4), object(5)
memory usage: 1.1+ MB


In [23]:
grouped = review_df.groupby('business_id')

In [26]:
business_ids, reviews = [], []
for name, group in grouped:
    business_ids.append(name)
    reviews.append(group.reset_index(drop=True))

In [27]:
reviews[0]

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,7o55X1bjinjaqR4_JbSO1w,8ibBR_p6xbY7Q8bymY3WVQ,--I7YYLada0tSLkORTHb5Q,4,1,1,1,"Had to get my wing fix, I like dry rubs on win...",2016-06-12 00:44:09
1,Mo4LuLt9X9EavM5zxfbQaQ,krKgjALhu5JdXVdJT9P47Q,--I7YYLada0tSLkORTHb5Q,5,1,1,1,"I have eaten here three times so far, and I ha...",2018-08-24 00:15:56
2,_Sc1HB5cz7cjIMkeYlQ4XQ,-DGtYtvbNVchc3GBgwlN3A,--I7YYLada0tSLkORTHb5Q,5,0,0,0,Had a great dinner here tonight. I asked the w...,2017-06-17 21:40:45
3,Vk65OyxdL01jzEh1z9RFVA,4d3TN2iaG474DfK3u6U2Ew,--I7YYLada0tSLkORTHb5Q,4,0,0,0,I am a truck driver have to sit a day Great p...,2015-06-21 18:28:45
4,hGttzBbw1cECvUbX5db7dg,GssXeSRb9ANv6HGBTH-N4A,--I7YYLada0tSLkORTHb5Q,5,1,0,0,One of the best sports bars around. The food i...,2017-01-28 21:19:05
...,...,...,...,...,...,...,...,...,...
95,5UO0-fdMmE_6ufkBlcwUUg,nfGC_3F7-urw7FirVRMcdA,--I7YYLada0tSLkORTHb5Q,2,2,0,0,"This place is okay. I don't know, it doesn't r...",2018-04-06 20:36:10
96,OXdzTX82GF6y4SGkNUkhTA,_MjgqPR1pvDnZZ6wCwabBQ,--I7YYLada0tSLkORTHb5Q,2,2,0,0,"So, after hearing what others had to say about...",2014-11-11 19:13:38
97,p9mipk44Q9aGbJkcayZTzA,M2FMa5OSTtdBgNjc6lEf2A,--I7YYLada0tSLkORTHb5Q,5,0,0,0,Happy Moose is consistently delicious. Their f...,2018-10-27 19:05:19
98,uhytYTmgSCcb4OZwqoBswg,aS3I98aPnbP--vIZlm4PJg,--I7YYLada0tSLkORTHb5Q,5,0,0,0,Today was my first time at Happy Moose. Upon e...,2017-06-17 04:58:47


In [88]:
reviews[0].describe()

,stars,useful,funny,cool
count,100.000000,100.000000,100.000000,100.000000
mean,3.470000,0.370000,0.210000,0.220000
std,1.359256,0.661419,0.498381,0.483673
min,1.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,0.000000
50%,4.000000,0.000000,0.000000,0.000000
75%,5.000000,1.000000,0.000000,0.000000
max,5.000000,3.000000,2.000000,2.000000


In [30]:
reviews[0]["text"][0]

"Had to get my wing fix, I like dry rubs on wings they had 3 different types couldn't choose between 6 pepper, smoked and cajun style so I just got all three. \r\nAll were great but if I needed to choose I would go with the 6 pepper, along with incredible craft beer offered makes this place a great stopping place while passing threw !"

In [34]:
agg_revs = '\n\n'.join([s for s in reviews[0]['text']])
print(agg_revs)

Had to get my wing fix, I like dry rubs on wings they had 3 different types couldn't choose between 6 pepper, smoked and cajun style so I just got all three. 
All were great but if I needed to choose I would go with the 6 pepper, along with incredible craft beer offered makes this place a great stopping place while passing threw !

I have eaten here three times so far, and I have walked away very pleased. Service has been great and highly recommend the burgers and boneless wings, both are the best I have ever had at a restaurant. Great selection of on tap beers as well. Give it a try.

Had a great dinner here tonight. I asked the waitress for a suggestion and she was spot on. Service was good employees were friendly and warm. Salads are fresh and terrific and overloaded with ingredients.

I am a truck driver have to sit a day  Great place ... Great wings ... Great waitress... But get a Sunday liquor license please my company comes here a lot

One of the best sports bars around. The foo

In [35]:
len(agg_revs.split())

8302

In [37]:
clean_punc = re.compile('[^a-zA-Z0-9\s]')
print(clean_punc.sub('', reviews[0]["text"][0].lower()))

had to get my wing fix i like dry rubs on wings they had 3 different types couldnt choose between 6 pepper smoked and cajun style so i just got all three 
all were great but if i needed to choose i would go with the 6 pepper along with incredible craft beer offered makes this place a great stopping place while passing threw 


In [40]:
# finding number of words
words = clean_punc.sub('', agg_revs.lower()).split()
print(words[:50])
print(len(words))

['had', 'to', 'get', 'my', 'wing', 'fix', 'i', 'like', 'dry', 'rubs', 'on', 'wings', 'they', 'had', '3', 'different', 'types', 'couldnt', 'choose', 'between', '6', 'pepper', 'smoked', 'and', 'cajun', 'style', 'so', 'i', 'just', 'got', 'all', 'three', 'all', 'were', 'great', 'but', 'if', 'i', 'needed', 'to', 'choose', 'i', 'would', 'go', 'with', 'the', '6', 'pepper', 'along', 'with']
8270


Average review length is about 83 words so final summary should be 100 to 200 words

In [140]:
summarizer = transformers.TFAutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = transformers.AutoTokenizer.from_pretrained("t5-base")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [44]:
inputs = tokenizer("summarize: " + agg_revs, return_tensors="tf", max_length=512)
outputs = summarizer.generate(inputs["input_ids"], max_length=200, min_length=100, length_penalty=2.0, num_beams=4, early_stopping=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [46]:
print(tokenizer.decode(outputs[0]))

<pad> i would give this place a 3 for food, but it's not necessarily bad. Just average. the house seasoned wings were perfect, but the real star was the Irish wing. i would recommend you stop at least once to try it out, especially if there is a band. i would give it a 4 for food, but it's not necessarily bad. Just average, but it's your typical, everyday bar food.


In [51]:
len(tokenizer.decode(outputs[0]))

367

In [49]:
summarizer2 = transformers.pipeline("summarization")

No model was supplied, defaulted to t5-small (https://huggingface.co/t5-small)


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [50]:
output2 = summarizer2(agg_revs, max_length=200, min_length=100, do_sample=False)
print(output2)

Token indices sequence length is longer than the specified maximum sequence length for this model (11027 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'Moose is a sports bar with a lot of burgers and fries . it\'s a great place to eat, but it is not a bad place to go back . the restaurant has a nice atmosphere and a good atmosphere . a burger is great, but the restaurant is ok, and the staff are friendly . "we\'re going to be back," says a friend of the restaurant\'s owner .'}]


In [54]:
print(output2[0]['summary_text'])

Moose is a sports bar with a lot of burgers and fries . it's a great place to eat, but it is not a bad place to go back . the restaurant has a nice atmosphere and a good atmosphere . a burger is great, but the restaurant is ok, and the staff are friendly . "we're going to be back," says a friend of the restaurant's owner .


In [56]:
len(output2[0]['summary_text'].split())

69

In [64]:
inputs3 = tokenizer("summarize: " + agg_revs, return_tensors="tf", max_length=None, truncation=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (11027 > 512). Running this sequence through the model will result in indexing errors


In [67]:
inputs["input_ids"]

<tf.Tensor: shape=(1, 512), dtype=int32, numpy=
array([[21603,    10, 10118,    12,   129,    82,     3,  3108,  2210,
            6,    27,   114,  2192,  9641,     7,    30, 13943,    79,
          141,   220,   315,  1308,  2654,    31,    17,   854,   344,
          431,  5270,     6,     3, 24198,    11,   212,  6959,   869,
           78,    27,   131,   530,    66,   386,     5,   432,   130,
          248,    68,     3,    99,    27,   906,    12,   854,    27,
          133,   281,    28,     8,   431,  5270,     6,   590,    28,
         3904,  5449,  6061,  1860,   656,    48,   286,     3,     9,
          248, 10847,   286,   298,  5792,     3,    17, 13296,     3,
           55,    27,    43, 16929,   270,   386,   648,    78,   623,
            6,    11,    27,    43,     3,  6412,   550,   182,  5010,
            5,  1387,    65,   118,   248,    11,  1385,  1568,     8,
            3,  9079,     7,    11,  6817,   924, 13943,     6,   321,
           33,     8,   200, 

# Testing extractive summarizers to reduce input size

In [85]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.kl import KLSummarizer

In [127]:
clean_revs = re.sub('(\.)\.+', ',', agg_revs)  # replaces 2 or more periods with a comma

In [128]:
parser = PlaintextParser.from_string(clean_revs, Tokenizer('english'))

In [129]:
lex_rank_summarizer = LexRankSummarizer()
lexrank_summary = lex_rank_summarizer(parser.document,sentences_count=32)

In [130]:
for sentence in lexrank_summary:
    print(sentence)

Service has been great and highly recommend the burgers and boneless wings, both are the best I have ever had at a restaurant.
But good.
Good, not great.
Very good.
There was like no sauce on it.
Very good!
Last night we went in and had the best service we have ever had there.
This was nice to see as they weren't very busy.
Not this time.
When I asked what beer the server would recommend, she chose one for me that was perfect and I will order from now on.
We will be back again, but if this continues to be the service we get I'm afraid we'll have to go somewhere else.
The wings are also very good.
Did not like at all.
Food good!
All of the food has been great.
The service was good and friendly, the food however was not.
I liked the place but I'll never go back if I can't drink as much as I'd like.
Service was good.
Great food.
The fries were also really good.
Great food and service!
The wings were just ok and the bill was huge for what we had.
Very good bar food!
It wasn't.
Great friend

In [131]:
start = time.time()
lsa_summarizer=LsaSummarizer()
lsa_summary= lsa_summarizer(parser.document,32)
end = time.time()
for sentence in lsa_summary:
    print(sentence)
print("\n\nTime taken: {0:.3f} s".format(end-start))

Not many places have an area that converts from a stage at night to tables during the day.
I would recommend you stop at least once to try it out, especially if there is a band.
We got boneless wings/fritters and they tasted like dirty fryer oil, as did the breaded chicken sandwiches.
It came with pita bread like Rocknes salads do, but mine was on the stale side.
They serve Pepsi products (I prefer Coke) but that wasn't enough to dampen my spirits.
Overall, great place to stop by if you're looking for a well seasoned meal, good drinks and pleasant staff.
As a note, I give stars based on how well the establishment accomplishes what it has set out to do.
They have a great beer list comprised of about 10 craft draft beers and seasonal bottles.
Two bartenders and only 10 of us at the bar but had to repeatedly wait for service.
Normally we like to leave at least 20% for a tip and typically give more, especially when service deserves it.
My friends and I were cut off after a few beers despit

In [132]:
start = time.time()
luhn_summarizer=LuhnSummarizer()
luhn_summary=luhn_summarizer(parser.document,sentences_count=32)
end = time.time()
for sentence in luhn_summary:
    print(sentence)
print("\n\nTime taken: {0:.3f} s".format(end-start))

Had to get my wing fix, I like dry rubs on wings they had 3 different types couldn't choose between 6 pepper, smoked and cajun style so I just got all three.
All were great but if I needed to choose I would go with the 6 pepper, along with incredible craft beer offered makes this place a great stopping place while passing threw !
Service has been great and highly recommend the burgers and boneless wings, both are the best I have ever had at a restaurant.
I am a truck driver have to sit a day  Great place , Great wings , Great waitress, But get a Sunday liquor license please my company comes here a lot
I'm generally not a harsh critic for many things, if there is effort involved I wouldn't be quick to judge on a first visit basis, but waiting almost 45 minutes just to be approached and then asked what we would like to drink is lousy.
My husband and I have been here twice, the first time being the day they opened and the second time, exactly two weeks later.
Service was very prompt and u

In [92]:
kl_summarizer=KLSummarizer()
kl_summary=kl_summarizer(parser.document,sentences_count=32)
for sentence in kl_summary:
    print(sentence)

Give it a try.
Had a great dinner here tonight.
Me and my girlfriend got our drinks after 45 minutes and ordered our food.
We also got the fried pickles.
As for the remark about the decor being sparse...It's a sports bar (not a gastro pub)...
Never asked if we needed anything.
The chicken was dry.
Both the service and the food were  exceptionally good.
The staff was nice.
Atmosphere smells like what it was a bar.
The service was good and friendly, the food however was not.
We ordered a Cuban and a chicken parm sandwich.
It was fantastic.
The fries were also really good.
This was a good meal for my boyfriend and I and for 24.99 we will definitely be back.
My companions had beer.
The wings were just ok and the bill was huge for what we had.
It was 59 cent wing night and the wings were a great quality.
Overall not good food and not worth the price.
The music was just a bit loud for my old ears.
We brought kids during the day and felt it was kid friendly.
Fresh cut fries were pretty good t

In [134]:
lsa_text = ' '.join([str(s) for s in lsa_summary])
print(len(clean_punc.sub('', lsa_text).split()))

589


In [135]:
luhn_text = ' '.join([str(s) for s in luhn_summary])
print(len(clean_punc.sub('', luhn_text).split()))

1079


In [147]:
# greedily choosing sentences such that total tokens is less than 512
lsa_text = []
wordlen = 0
for sentence in lsa_summary:
    s = str(sentence)
    w = len(s.split())
    if wordlen + w < 450:  # some buffer for sub-word tokenization and punctuation
        lsa_text.append(s)
        wordlen += w
lsa_text = ' '.join(lsa_text)
print(lsa_text)

Not many places have an area that converts from a stage at night to tables during the day. I would recommend you stop at least once to try it out, especially if there is a band. We got boneless wings/fritters and they tasted like dirty fryer oil, as did the breaded chicken sandwiches. It came with pita bread like Rocknes salads do, but mine was on the stale side. They serve Pepsi products (I prefer Coke) but that wasn't enough to dampen my spirits. Overall, great place to stop by if you're looking for a well seasoned meal, good drinks and pleasant staff. As a note, I give stars based on how well the establishment accomplishes what it has set out to do. They have a great beer list comprised of about 10 craft draft beers and seasonal bottles. Two bartenders and only 10 of us at the bar but had to repeatedly wait for service. Normally we like to leave at least 20% for a tip and typically give more, especially when service deserves it. My friends and I were cut off after a few beers despit

In [150]:
# greedily choosing sentences such that total tokens is less than 512
luhn_text = []
wordlen = 0
for sentence in luhn_summary:
    s = str(sentence)
    w = len(s.split())
    if wordlen + w < 450:  # some buffer for sub-word tokenization and punctuation
        luhn_text.append(s)
        wordlen += w
luhn_text = ' '.join(luhn_text)
print(luhn_text)

Had to get my wing fix, I like dry rubs on wings they had 3 different types couldn't choose between 6 pepper, smoked and cajun style so I just got all three. All were great but if I needed to choose I would go with the 6 pepper, along with incredible craft beer offered makes this place a great stopping place while passing threw ! Service has been great and highly recommend the burgers and boneless wings, both are the best I have ever had at a restaurant. I am a truck driver have to sit a day  Great place , Great wings , Great waitress, But get a Sunday liquor license please my company comes here a lot I'm generally not a harsh critic for many things, if there is effort involved I wouldn't be quick to judge on a first visit basis, but waiting almost 45 minutes just to be approached and then asked what we would like to drink is lousy. My husband and I have been here twice, the first time being the day they opened and the second time, exactly two weeks later. Service was very prompt and u

In [148]:
output3 = summarizer2(lsa_text, max_length=400, min_length=200, do_sample=False)
print(output3[0]['summary_text'])

if you're looking for a well seasoned meal, good drinks and pleasant staff, I give stars based on how well the establishment accomplishes what it has set out to do . two bartenders and only 10 of us at the bar were cut off after a few beers despite having THREE designated drivers among our 7 person party . four days later we receive an alert from our credit card company questioning us of a large tip applied to the amount . it's good to see a 'home', '',, ......... ... and ... .. 'i think it'd like to a good . "the . the s e a great beer list and the 'the beer & the . and that . but a place in a bar and a group of 7 people at a local .


In [151]:
output3 = summarizer2(luhn_text, max_length=400, min_length=200, do_sample=False)
print(output3[0]['summary_text'])

service has been great and highly recommend the burgers and boneless wings, both are the best I have ever had at a restaurant . my husband and I have been here twice, the first time being the day they opened and the second time, exactly two weeks later . service was very prompt and unlike the other review it seems obvious to us you can sit at the bar or wait to be seated by the hostess since there's a host's stand . the server took our drink order, 10 minutes later my ... ... and . 'the burger burger and burger, along with incredible craft beer . it is a 'good and a good burger with a great and if it was if they were 'n a day and 'how to a first, and the other . to get a Sunday liquor license, but the 'best, the .


In [184]:
inputs = tokenizer("summarize: " + lsa_text, return_tensors="tf", max_length=512)
outputs = summarizer.generate(inputs["input_ids"], min_length=100, max_length=250, eos_token_id=None, length_penalty=0.5, num_beams=6, early_stopping=True, repetition_penalty=2.0)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

great place to stop by if you're looking for a well-seasoned meal, good drinks and pleasant staff. boneless wings/fritters tasted like dirty fryer oil, as did the breaded chicken sandwiches. two bartenders and only 10 of us at their pub but had repeatedly wait in line for service; they have now lost us as customers & our family of 6 that we would take there again despite having designated drivers among our 7 person party!


In [186]:
start = time.time()
inputs = tokenizer("summarize: " + luhn_text, return_tensors="tf", max_length=512)
outputs = summarizer.generate(inputs["input_ids"], min_length=100, max_length=250, eos_token_id=None, length_penalty=2.0, num_beams=6, early_stopping=True, repetition_penalty=2.0)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
end = time.time()
print("\n\nTime taken: {0:.3f} s".format(end-start))

i've been to this place twice, the first time being the day they opened and second time exactly two weeks later. service was very prompt and unlike other reviews it seems obvious to you can sit at bar or wait for hostesses since there is a hostessed stand - no one thought to sweep under table or wipe chairs before sitting new group! food is good but not out of the ordinary and much like an applebeer’s, TGIFridays, or Chili'?


Time taken: 27.515 s


# Final model

In [189]:
review_df = pd.read_csv("reviewSelected100.csv")
grouped = review_df.groupby('business_id')
business_ids, reviews = [], []
for name, group in grouped:
    business_ids.append(name)
    reviews.append(group.reset_index(drop=True))

In [191]:
luhn_summarizer=LuhnSummarizer()  # initializing extractive summarizer
# initializing abstractive summarizer
summarizer = transformers.TFAutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = transformers.AutoTokenizer.from_pretrained("t5-base")

for i in range(len(business_ids)):
    biz_id = business_ids[i]
    temp_df = reviews[i]
    temp_revs = '\n\n'.join([s for s in temp_df['text']])
    temp_revs = re.sub('(\.)\.+', ',', temp_revs)
    
    
    start = time.time()
    print("Starting for business id:", biz_id)
    
    # extractive summarization
    temp_parser = PlaintextParser.from_string(temp_revs, Tokenizer('english'))
    temp_luhn_summary=luhn_summarizer(temp_parser.document,sentences_count=32)
    
    # greedily choosing sentences such that total tokens is less than 512
    temp_luhn_text = []
    wordlen = 0
    for sentence in temp_luhn_summary:
        s = str(sentence)
        w = len(s.split())
        if wordlen + w < 450:  # some buffer for sub-word tokenization and punctuation
            temp_luhn_text.append(s)
            wordlen += w
    temp_luhn_text = ' '.join(temp_luhn_text)
    
    # abstractive summarizer model
    temp_inputs = tokenizer("summarize: " + temp_luhn_text, return_tensors="tf", max_length=512)
    temp_outputs = summarizer.generate(temp_inputs["input_ids"], min_length=100, max_length=250, 
                                       eos_token_id=None, length_penalty=2.0, num_beams=6, 
                                       early_stopping=True, repetition_penalty=2.0)
    temp_summary = tokenizer.decode(temp_outputs[0], skip_special_tokens=True)
    end = time.time()
    print("Done! Time taken: {0:.3f} s".format(end-start))
    print("Generated summary:")
    print(temp_summary)
    
    # saving output
    with open('generated_reviews.csv', 'a') as fileobj:
        writer = csv.writer(fileobj)
        writer.writerow([biz_id, temp_summary])
    
    print("\nSummary saved successfully.\n\n")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Starting for business id: --I7YYLada0tSLkORTHb5Q
Done! Time taken: 31.503 s
Generated summary:
i've been to this place twice, the first time being the day they opened and second time exactly two weeks later. service was very prompt and unlike other reviews it seems obvious to you can sit at bar or wait for hostesses since there is a hostessed stand - no one thought to sweep under table or wipe chairs before sitting new group! food is good but not out of the ordinary and much like an applebeer’s, TGIFridays, or Chili'?

Summary saved successfully.


Starting for business id: -7XWJYkutqhIxLen7Grg1g
Done! Time taken: 30.349 s
Generated summary:
prince sushi is one of my favourite sushi places in the GTA. it's located at the AMC Theatre plaza and is small in size - with about 15 smaller tables! food is alright, menu is okay, and prices are good too... but having said that i found this place to be just regular!! they have a great selection of rolls, grilled meat and sashimi as well as a goo

Done! Time taken: 32.378 s
Generated summary:
in my 50+ years of life i've had the opportunity to eat at many so called ‘chinese buffets’ from left coast, right coast and even one in Pittsburgh. I was very excited when Tokyo Grill opened in the University City area and the first couple times that we ate there were very pleased, especially with their sushi bar - and it gets busy FAST compared to other Asian buffets a little cheaper if you come just before lunch/dinner.

Summary saved successfully.


Starting for business id: 9Xm2GfG8Rnbb1_CmXyrm3g
Done! Time taken: 35.475 s
Generated summary:
the wings are the tiniest chicken wings I ever had, with absolutely no sauce, and they only served six tiny little wings and two pieces of carrots for nine dollars. The upstairs bar tender was to busy texting on her phone instead than helping us - so we called in a pick up order and were told the wings would be 30 minutes, which is alright but we'd rather not wait, so cancelled that part of our ord

Done! Time taken: 31.525 s
Generated summary:
waffle house is one of the cleaner and newer feeling Waffle Houses. food is amazing, first time I visited they thought "that're not enough" server was rude and stated with attitude and annoyance as to why we didn’t say anything when we were waiting for something good... but she said "ok" and walked off after awkward silence a waitress made at least half my order while other lady got confused between orders of another customer....

Summary saved successfully.


Starting for business id: IBYExgJ6jURNJOk_5uSrKg
Done! Time taken: 34.717 s
Generated summary:
the sampler included cold practically still frozen edamama and items that seriously looked like they came straight out of. box in my grocery store - then called out your name or the order and expecting you go fetch if you want to eat there 'at the table' instead of pointing me in direction of the items and watching him get up to retrieve them for myself, my server told us not too worry about

Done! Time taken: 33.780 s
Generated summary:
if you take the time to seek out this restaurant you will not be disappointed! every sandwich on menu is amazing and you have must try the house special "Sloppy Joseph" (don't pass up on the jalapeos and queacho) it'll be a great place for a quick lunch or dinner after a long day at work, but there are other things on the menu that we still haven’t tried yet. don'd eat here again unless you were in town for 2 weeks

Summary saved successfully.


Starting for business id: NV5Q8ZhDSh_oI8ZOnn72Lw
Done! Time taken: 31.655 s
Generated summary:
the fries are hot and limp the way hand cut burgers should be, but they're not worth waiting for when you have as little time left as I do. of course, my family just had an early dinner and his brother was telling him that the Burger Factory has funnel cakes and that we should go out for it as group dessert - so that is a good omen if you are into burger choices... some of which look to being larger such l

...

Done! Time taken: 31.281 s
Generated summary:
i've been there 6 times (no car closest food chain) as I work around the corner. they made a few mistakes including giving me wrong burger (I ordered the lunch special 5$ double-baked burger with drink combo) when ordering the waffle house special the cook seemed to be upset over my disrupting his convo he finished it literally in 2 minutes. when paying for the special she deducted.60c when asked whether that is how much hash browns cost she replied rudely and left

Summary saved successfully.


Starting for business id: ZhACHVw9k438Z3i-Gw9BHA
Done! Time taken: 27.809 s
Generated summary:
banh mi boys is opening in my hood and I'm glad they did so. everything had so much flavor yet it was well balanced - nothing too spicy but still had heat, texture of the bread and texture of both the bao and the fries were just beautiful gluttonous food i'll try to make them at home! you can get a traditional bang mi elsewhere for cheaper but those places

Done! Time taken: 29.053 s
Generated summary:
i've always had great experience with Hi Tide and whenever I have my two year old in tow, they really accommodate us and make sure we had options for him or her to eat. this one was unique because you order from the menu at counter but don’t see ingredients when your ordering and aren;t watching them make it - so you dont watch them make it! after another 10-15 minutes the manager came over and let us know that the ticket didnâ€TMt make it to kitchen and kept saying

Summary saved successfully.


Starting for business id: e-YnECeZNt8ngm0tu4X9mQ
Done! Time taken: 27.825 s
Generated summary:
the servers took forever to come take our orders and i thought it was a bit ridiculous. as we were eating, service suffered greatly and at times we had refills our own water glasses & RICE - yes I said "rice" was hard to get if you came with 4 people trying all their food on one little grill... but that'd be a bummer. there is apparently nothing authentic

Done! Time taken: 29.224 s
Generated summary:
i went to Jack Astors on wed and it wasn't too busy some reason the service was so slow and their food was horrible. since all guests have not arrived as of yet the hostesses suggested that we take the table for 5 and when another tables opens up we can move over to that area - I almost always get one stopping by to ask whether they've ordered something, only to find out it is the next door or not even near our own! after eating had it pushed back to front of table,

Summary saved successfully.


Starting for business id: mF2EW3twSrFPmT_RVV1-Qg
Done! Time taken: 29.186 s
Generated summary:
if you sit in the regular area, service has no way to know what you will be ordering. they do not offer a children's menu or child-sized portions on either the sushi or the regular menu but it is surprisingly filling... especially for younger diners! "i was annoyed that this wasn’t made clear with my initial question," says one reviewer of the restaurant 

Done! Time taken: 26.462 s
Generated summary:
the flower shop is on the small side and their inventory is a little limited because of this. "my girl was upset that I never give her flowers, so...i came here and was very impressed by what they have to offer," says cnbc's sarah mccartney jr - owner of the shop whose business is out of business thursday night edward mccartney & son.

Summary saved successfully.


Starting for business id: rv9T1qGHbh9eXqFpKEPy_A
Done! Time taken: 25.572 s
Generated summary:
if you love chinese food and don't know what it is like or should be, do not come here. instead of large plates of food and free noodles, they give you half the food, no more noodle things - still charge the same amount as at your typical cheap Chinese place in north Scottsdale area! "i have never had china food in a while and don’t think why I would bother coming up this far north to get it"

Summary saved successfully.


Starting for business id: s7keQ0-OQ5Li_IKGKVTarw
Done! Time take

Done! Time taken: 30.940 s
Generated summary:
cna's kelly wallace was treated like someone getting nails done should be by one of the other nail ladies. they cut two cuticles open and bleeding while she watched another girl come in and be treated like that - wallace! "extreme" polish is on my toenails and now it'll actually get what I want when spending money, wallace says at the salon he hated neon polish i had asked them for redo my nails twice but still layering on now polish 

Summary saved successfully.


Starting for business id: x3LoVFuzVIPK_J-kDZORdg
Done! Time taken: 31.740 s
Generated summary:
la unica is a Mexican restaurant in north carolina. the food was decent, but the waitresses weren't very talkative - reviewer says it'll be better next time if they would have handled things differently i will not be returning anytime soon he says: "you can have best food in town but unless you can get it right or hot or in an timely manner then I'll go find someone who won’T get me med

# Additional cleaning of summaries
We will attempt to add truecasing and spell checking

In [264]:
summary_df = pd.read_csv('generated_reviews.csv')
summary_df.head()

,business_id,summary
0,--I7YYLada0tSLkORTHb5Q,"i've been to this place twice, the first time ..."
1,-7XWJYkutqhIxLen7Grg1g,prince sushi is one of my favourite sushi plac...
2,0Rni7ocMC_Lg2UH0lDeKMQ,"barros pizza can get pretty busy, so if you're..."
3,0kPm1zEpeXFRg8D2phqgCQ,i'm conflicted with giving this place one star...
4,1Fpk8ibHhZYnCw8fnGny8w,the entire main level of this restaurant makes...


In [215]:
nlp = spacy.load('en_core_web_sm')

In [262]:
def add_capitalization(inpt):
    """
    Need a function to do this because nltk and .capitalize() reduce everything to lowercase, want to preserve original upper cases like in acronyms
    """
    doc = nlp(inpt)
    tagged_sent = [(w.text, w.tag_) for w in doc]
    capitalized = []
    for i in range(len(tagged_sent)):
        w, t = tagged_sent[i]
        if t in ["NNP", "NNPS"] or w=='i' or i==0 or (i>0 and tagged_sent[i-1][0] in '.?!;'):
            capitalized.append(w[0].upper() + w[1:])  # not using .capitalize() because other capitalization is lost
        else:
            capitalized.append(w)
    res = re.sub(" (?=[\.,'!?:;])", "", ' '.join(capitalized))
    return res

In [255]:
add_capitalization(summary_df['summary'][0])

"I've been to this place twice, the first time being the day they opened and second time exactly two weeks later. Service was very prompt and unlike other reviews it seems obvious to you can sit at bar or wait for hostesses since there is a hostessed stand - no one thought to sweep under table or wipe chairs before sitting new group! Food is good but not out of the ordinary and much like an applebeer ’s, TGIFridays, or Chili'?"

In [265]:
summary_df['summary'] = summary_df.apply(lambda x: add_capitalization(x['summary']), axis=1)
summary_df.head()

,business_id,summary
0,--I7YYLada0tSLkORTHb5Q,"I've been to this place twice, the first time ..."
1,-7XWJYkutqhIxLen7Grg1g,Prince Sushi is one of my favourite sushi plac...
2,0Rni7ocMC_Lg2UH0lDeKMQ,"Barros pizza can get pretty busy, so if you're..."
3,0kPm1zEpeXFRg8D2phqgCQ,I'm conflicted with giving this place one star...
4,1Fpk8ibHhZYnCw8fnGny8w,The entire main level of this restaurant makes...


In [266]:
summary_df.to_csv('cleaned_reviews.csv', index=False)